In [1]:
import MySQLdb
import pandas as pd
import numpy as np
from datetime import datetime

In [5]:
hc_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_2_1_herbal_comp_gold_drug_se_sim_mat_predict.tsv", index_col=0)
hc_str = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_2_1_herbal_comp_gold_drug_se_sim_mat_predict.tsv", index_col=0)

In [8]:
hc_ids = hc_se.index.tolist()
print len(hc_ids)

100


In [2]:
#접속하기
db_handle = MySQLdb.connect('localhost', 'root','comsysbio', 'COCONUTv5')
#cur = db_handle.cursor(MySQLdb.cursors.DictCursor) # query를 통해 데이터를 얻을 때 python의 dict로 얻기 위한 방법
cursor = db_handle.cursor()

In [15]:
"{}".format(tuple(hc_ids))

'(5525, 1020, 732, 9210, 926, 4020, 86025, 70460, 82835, 79944, 79942, 79938, 79941, 1530, 1531, 79939, 153, 8019, 4286, 70319, 8885, 3761, 119, 139, 70410, 70661, 4521, 1878, 622, 543, 9471, 9926, 85183, 3005, 10195, 7874, 59654, 50263, 80081, 80082, 80097, 80164, 2969, 76198, 58454, 61885, 8739, 562, 77956, 614, 40553, 773, 65817, 71038, 70458, 5529, 1166, 71664, 8024, 837, 416, 79982, 7475, 83229, 89181, 760, 10086, 31743, 2682, 823, 79684, 845, 752, 70473, 7859, 78876, 1001, 666, 1574, 58258, 7996, 1167, 71205, 43066, 9670, 141, 243, 1331, 53060, 70973, 8012, 61516, 70924, 77084, 7638, 70692, 71009, 85310, 1343, 1529)'

In [18]:
# herbal compound와 관련있는 gene id 뽑기
query1 = "SELECT compID, geneID, reference FROM Compound_Gene where compID in {} and (organism=\"human\" or organism=\"homo sapiens\")".format(tuple(hc_ids))
cursor.execute(query1); #쿼리 날리는 거
results = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [41]:
np_results = np.array(results)

In [43]:
df_results = pd.DataFrame({'compID' : np_results[:, 0], 'geneID' : np_results[:, 1], 'reference' : np_results[:, 2]})

In [122]:
print len(df_results)
print len(df_results.drop_duplicates('compID')), len(df_results.drop_duplicates('geneID'))
df_results = df_results[['compID', 'geneID']].drop_duplicates()
print len(df_results)
df_results.head(1)

35808
66 8342
18525


,compID,geneID
0,119,283


In [124]:
# gene 정보가 없는 compound들
a = [int(x) for x in df_results['compID'].drop_duplicates().tolist()]
hc_se.index[~hc_se.index.isin(a)]

Int64Index([70460, 82835, 79944, 79941, 70319,  8885, 70410, 70661, 85183,
            59654, 50263, 80081, 76198, 58454, 61885, 40553, 65817, 71038,
            70458, 71664, 89181, 31743, 79684, 70473, 71205, 53060, 70973,
            61516, 70924, 77084,  7638, 70692, 71009, 85310],
           dtype='int64')

In [125]:
hc_ids2 = hc_se.index[hc_se.index.isin(a)].tolist()

In [90]:
# geneId에 해당하는 entrezID와 uniprotID 뽑기
gene_ids = [int(x) for x in df_results.geneID.drop_duplicates().tolist()]
query2 = "SELECT geneID, entrezID, uniprotID FROM Gene where geneID in {}".format(tuple(gene_ids))
cursor.execute(query2); #쿼리 날리는 거
results2 = cursor.fetchall() # cursor.fetchone()은 tuple, cursor.fetchall()은 list of tuples을 리턴한다.

In [97]:
results2[0]

(1L,
 '2147',
 'P00734 / A0N064|A8IM47|C8BKD1|E7FAN5|G1NEM6|G1PXB6|G1TB36|G3GYJ4|G3V843|H0WQ57|H0ZJZ8|H2Q3I2|I3M5J3|J9NSF9|K4G0I9|M3WSI8|M3XJR1|P00734|P00735|P19221|Q19AZ8|Q3TJ94|Q4QR53|Q5FVW1|Q5NVS1|Q5R537|Q7SXH8|Q804W7|Q91001|U3J210|U3JUU7 / P00734/B2R7F7/Q69EZ7/B4E1A7/Q9UCA1/Q53H04/Q7Z7P3/Q4QZ40/Q53H06')

In [98]:
np_results2 = np.array(results2)

In [100]:
np_results2[0]

array(['1', '2147',
       'P00734 / A0N064|A8IM47|C8BKD1|E7FAN5|G1NEM6|G1PXB6|G1TB36|G3GYJ4|G3V843|H0WQ57|H0ZJZ8|H2Q3I2|I3M5J3|J9NSF9|K4G0I9|M3WSI8|M3XJR1|P00734|P00735|P19221|Q19AZ8|Q3TJ94|Q4QR53|Q5FVW1|Q5NVS1|Q5R537|Q7SXH8|Q804W7|Q91001|U3J210|U3JUU7 / P00734/B2R7F7/Q69EZ7/B4E1A7/Q9UCA1/Q53H04/Q7Z7P3/Q4QZ40/Q53H06'], 
      dtype='|S28074')

In [232]:
df_results2 = pd.DataFrame({'geneID' : np_results2[:, 0], 'entrezID' : np_results2[:, 1], 'uniprotID' : np_results2[:, 2]})

In [199]:
df_results2.head(1)

,entrezID,geneID,uniprotID
0,2147,1,P00734 / A0N064|A8IM47|C8BKD1|E7FAN5|G1NEM6|G1...


In [200]:
print len(df_results2)
print len(df_results2.drop_duplicates('entrezID')), len(df_results2.drop_duplicates('geneID')) # entrez는 null 인것이 사라짐
print len(df_results2)
df_results2.head(1)

8342
8325 8342
8342


,entrezID,geneID,uniprotID
0,2147,1,P00734 / A0N064|A8IM47|C8BKD1|E7FAN5|G1NEM6|G1...


In [239]:
# uniprot ID를 row당 하나 오게 바꾸기
aa = df_results2['uniprotID'].str.split('/').apply(pd.Series, 1).stack()
aa.index = aa.index.droplevel(-1)
aa.name = 'uniprotID'
temp = df_results2.copy()
del temp['uniprotID']
bb = temp.join(aa)
cc = bb['uniprotID'].str.split('|').apply(pd.Series, 1).stack()
cc.index = cc.index.droplevel(-1)
cc.name = 'uniprotID'
temp = bb.copy()
del temp['uniprotID']
dd = temp.join(cc)

In [241]:
print len(dd)
print len(dd.drop_duplicates())

1897358
248687


In [267]:
dd.uniprotID = dd.uniprotID.str.lstrip().str.rstrip()

In [271]:
df_results3 = dd.drop_duplicates()

In [272]:
merged = pd.merge(left = df_results, right = df_results3, how='inner')

In [275]:
print len(df_results)
print len(df_results3)
print len(merged)
print len(merged.drop_duplicates())

18525
236703
595543
595543


In [351]:
print len(merged.drop_duplicates('uniprotID'))
merged.head(1)

236321


,compID,geneID,entrezID,uniprotID
0,119,283,4846,P29474


In [344]:
hc_seq_raw = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/new_predict/hc_seq_raw.tab")

In [352]:
hc_seq = pd.merge(left = merged, right=hc_seq_raw, left_on = 'uniprotID', right_on='Entry', how='inner')

In [362]:
hcsorted = hc_seq.sort_values(by=['compID', 'geneID'])

In [374]:
hc_seq[['entrezID', 'uniprotID', 'Sequence']].drop_duplicates().sort_values(by=['entrezID', 'Sequence']).to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/entrez_uniprot.tsv", sep='\t')

In [377]:
print len(hc_seq[['entrezID', 'uniprotID', 'Sequence']].drop_duplicates().sort_values(by=['entrezID', 'Sequence']))
print len(hc_seq.compID.drop_duplicates())

12081
66


In [376]:
hc_seq[['entrezID', 'uniprotID', 'Sequence']].drop_duplicates().sort_values(by=['entrezID', 'Sequence'])

,entrezID,uniprotID,Sequence
1514,10,P11245,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQ...
1518,10,A4Z6T7,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQ...
15455,100,F5GWI4,MAQTPAFDKPKVELHVHLDGSIKPETILYYGRRRGIALPANTAEGL...
15450,100,P00813,MAQTPAFDKPKVELHVHLDGSIKPETILYYGRRRGIALPANTAEGL...
19053,1000,P19022,MCRIAGALRTLLPLLAALLQASVEASGEIALCKTGFPEDVYSAVLS...
17394,10003,Q9Y3Q0,MAESRGRLYLWMCLAAALASFLMGFMVGWFIKPLKETTTSVRYHQS...
16147,10004,Q9UQQ1,MQWTKVLGLGLGAAALLGLGIILGHFAIPKKANSLAPQDLDLEILE...
23536,10005,O14734,MSSPQAPEDGQGCGDRGDPPGDLRSVLVTTVLNLEPLDEDLFRGRH...
25767,10006,B6VEX5,MAELQMLLEEEIPSGKRALIESYQNLTRVADYCENNYIQATDKRKA...
25768,10006,Q8IZP0,MAELQMLLEEEIPSGKRALIESYQNLTRVADYCENNYIQATDKRKA...


In [331]:
# 일단 GO를 위한 compID, geneID, entrezID 파일을 만든다.
hc_go = merged[['compID', 'geneID', 'entrezID']].drop_duplicates()
hc_gp = hc_go[hc_go.entrezID != 'null']

In [332]:
hc_go.head(1)

,compID,geneID,entrezID
0,119,283,4846


In [334]:
print len(hc_go.compID.drop_duplicates())

66


In [341]:
iid = merged.entrezID.drop_duplicates().tolist()
f = open("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/entrez_id.txt", 'w')
for item in iid:
    f.write("%s\r\n" % item)
f.close()

In [342]:
print len(iid)

8325


In [335]:
hc_go.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/4_3_herbal_compounds_gene.tsv", sep='\t', index=False)

In [276]:
len(merged.uniprotID.drop_duplicates())

236321

In [277]:
len(merged.uniprotID)

595543

In [284]:
tarSim = pd.read_excel("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/similarity_protein_gold_indep.xlsx", header=None)

In [280]:
tarSeq = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/new_predict/drug_target_seq.tsv")

In [286]:
tarSim.head(1)

,0,1,2,3,4,5,6,7,8,9,...,2299,2300,2301,2302,2303,2304,2305,2306,2307,2308
0,1.0,0.007301,0.007301,0.00921,0.00921,0.015303,0.015303,0.015303,0.015303,0.013242,...,0.012339,0.013388,0.005891,0.005685,0.013741,0.011477,0.012288,0.010557,0.015177,0.012273


In [285]:
print tarSim.shape
print tarSeq.shape

(2309, 2309)
(2309, 5)


In [287]:
print len(tarSeq.Target.drop_duplicates())

670


In [299]:
target_sequence = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/protein_seq.txt")

In [309]:
target_sequence.head()

,Entry,Sequence,Length
0,P19113,MMEPEEYRERGREMVDYICQYLSTVRERRVTPDVQPGYLRAQLPES...,662.0
1,Q9UI32,MRSMKALQKALSRAGSHCGRGGWGHPSRSPLLGGGVRHHLSEAAAQ...,602.0
2,P00488,MSETSRTAFGGRRAVPPNNSNAAEDDLPTVELQGVVPRGVNLQEFL...,732.0
3,P35228,MACPWKFLFKTKFHQYAMNGEKDINNNVEKAPCATSSPVTQDDLQY...,1153.0
4,P37059,MSTFFSDTAWICLAVPTVLCGTVFCKYKKSSGQLWSWMVCLAGLCA...,387.0


In [310]:
#tarSeq[~tarSeq.Target.isin(merged.uniprotID)]
kkk = merged[merged.uniprotID.isin(target_sequence.Entry)]

In [314]:
print len(kkk.uniprotID.drop_duplicates())
print len(merged.uniprotID.drop_duplicates())
print len(target_sequence.Entry.drop_duplicates())

1859
236321
2290


In [316]:
print len(kkk.compID.drop_duplicates())

65
